In [42]:
import os, sys
import gdown
import codecs
import json
import pprint
from PyPDF2 import PdfReader
import docx2txt
import wget


In [43]:
# !pip install gdown
# !pip install PyPDF2
# !pip install docx2txt
# !pip install wget


In [44]:
PATH_TO_COURSE_DATA = os.path.join("data")

PATH_TO_COURSES_TXT = os.path.join(PATH_TO_COURSE_DATA, "courses_txt")
PATH_TO_COURSE_JSON = os.path.join(PATH_TO_COURSE_DATA, "courses.json")

In [74]:
def load_from_txt(path : str, if_change_codec : bool = False):
    # http://alexei.stepanov.spb.ru/students/Abstracts/ClusterAlg.txt
    text = None
    f = None
    if if_change_codec:
        f = codecs.open(path, 'r', 'cp1251')
    else:
        f = codecs.open(path, 'r')
    is_read = False
    
    try:
        text = f.read()   # now the contents have been transformed to a Unicode string
        is_read = True
        file_type = "txt"
    except:
        print(f"Это не txt :( !!!")
    
    return text, is_read


def load_from_pdf(path : str):
    text = None
    is_read = False
    
    try:
        reader = PdfReader(path)
        text = reader.pages[0].extract_text()
        is_read = True
        file_type = "pdf"
    except:
        print(f"Это не pdf !!!")
    
    return text, is_read

def load_from_docx(path : str):
    # https://drive.google.com/uc?id=1OzogYQIQ8lf6HBYvM6wjhsGAcSN2TbQY
    text = None
    is_read = False
    
    try:
        text = docx2txt.process(path)
        is_read = True
        file_type = "docx"
    except:
        print(f"Это не docx !!!")
    
    return text, is_read




In [68]:
def convert_to_text(path : str):
    print(f"convert_to_text: output_path={path}")
    
    file_type = None    
    
    text, ok = load_from_txt(output_path)
    if not ok:
        text, ok = load_from_txt(output_path, if_change_codec=True)
    if not ok:
        text, ok = load_from_pdf(output_path)
    if not ok:
        text, ok = load_from_docx(output_path)
    
    if not ok:
        print(f"convert_to_text: [IS NOT DEFAULT] output_path={path}")
        return None
    out = open(path, 'w', encoding='utf-8')
    out.write(text)   # and now the contents h
    
    return file_type

In [69]:
# def download_course(link : str, course_short_name: str, output_path_to_dir=PATH_TO_COURSES_TXT):
#     print(f"download_course: {course_short_name}, url={link}")
#     output_path = os.path.join(output_path_to_dir, f"{course_short_name}.txt")
    
    
#     gdown.download(link, output_path, quiet=False, fuzzy=True, use_cookies=True)
    
#     file_type = change_codec(output_path)
#     print(f"{course_short_name} [{file_type}] ready: {link}")
    
    

In [70]:
def is_google_drive(link: str):
    ok = False
    trigger = "sharing"
    if link[-len(trigger):] == trigger:
        ok = True
    return ok
    

def is_txt_link(link : str):
    ok = False
    trigger = "txt"
    if link[-len(trigger):] == trigger:
        ok = True
    return ok
    
def is_pdf_link(link : str):
    ok = False
    trigger = "pdf"
    if link[-len(trigger):] == trigger:
        ok = True
    return ok

def is_htm_link(link : str):
    ok = False
    trigger = "htm"
    if link[-len(trigger):] == trigger:
        ok = True
    return ok
    

In [71]:
courses_json = json.load(open(PATH_TO_COURSE_JSON, "r", encoding="utf-8"))

In [72]:
ban_list = ["AdvComb", "NonClassL", "NonStandard", "Logic",
           "MachLPr", "NLPcourse", "PracticalRL", "PracticalDL", "SelfDriving", "SpeechCourse"]

In [73]:
for course_json in courses_json:
    url = course_json['description']
    short_name = course_json['short_name']
    field_of_knowledge = course_json['field_of_knowledge']
    if short_name in ban_list:
        continue
    
    
    if field_of_knowledge == "Алгебра" :
        continue
#     elif field_of_knowledge == "Алгебра"
    path_dir_to_save = os.path.join(PATH_TO_COURSES_TXT, field_of_knowledge)
    if not os.path.isdir(path_dir_to_save):
        os.makedirs(path_dir_to_save)
        
    output_path = os.path.join(path_dir_to_save, f"{short_name}.txt")
    if os.path.isfile(output_path):
        continue
    
    if is_google_drive(url):
        print(f"{short_name} скачан с googleDrive")
        gdown.download(url, output_path, quiet=False, fuzzy=True, use_cookies=True)
        
    elif is_txt_link(url):
        print(f"{short_name} скачан с txt")
        file_name = wget.download(url, path_dir_to_save)
        os.rename(file_name, output_path)
    elif is_pdf_link(url):
        print(f"{short_name} скачан с pdf")
        file_name = wget.download(url, path_dir_to_save)
        os.rename(file_name, output_path)
    elif is_htm_link(url):
        print(f"{short_name} скачан с htm")
        file_name = wget.download(url, path_dir_to_save)
        os.rename(file_name, output_path)
    else: 
        print(f"{short_name} не понял тип, пропустил( {url}")
        continue
        
        
    file_type = convert_to_text(output_path)
    print(f"{short_name} [{file_type}] ready: {url}")
    
    

Teichmull скачан с txt
100% [..............................................................] 659 / 659convert_to_text: output_path=data/courses_txt/Геометрия и топология/Teichmull.txt
Это не txt :( !!!
Teichmull [None] ready: http://alexei.stepanov.spb.ru/students/Abstracts/Teichmull.txt
TopComb скачан с googleDrive


TooManyRedirects: Exceeded 30 redirects.